In [1]:
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageMessage, AudioMessage
)

# 載入設定檔

import json
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [2]:
from aip import AipSpeech
import json
import time
import os
import wave
from pydub import AudioSegment

""" 你的 APPID AK SK """
APP_ID = '17610496'
API_KEY = '3HxD7AjTwLrjEdWzjno97lbl'
SECRET_KEY = 'rPBbzb3enl2EA0cK0n98zHktbWDqYPt8'

begin = time.time()
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)


# 读取文件
def get_file_content(filepath):
    with open(filepath, 'rb') as fp:
        return fp.read()

In [9]:
@handler.add(MessageEvent, message=AudioMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='沒事亂傳訊息，害我又要加班'))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('./audios/'+event.message.id+".aac", 'wb') as fd:           ####格式問題:m4a,wav,mp4都試過了
        for chunk in message_content.iter_content():
            fd.write(chunk)
            
    
            
#     # 识别本地文件
#     voicefile = event.message.id+".wav"
#     base = "./audios/"
#     # print(base+voicefile)
#     result = client.asr(get_file_content(base+voicefile), 'wav', 16000, {
#         'dev_pid': 1536,
#     })
#     # print(result)
#     print("Request time cost %f" % (time.time() - begin))
#     if result['err_no'] == 0:
#         ofile = "result_%s.txt" % (voicefile)
#         with open(ofile, "w", encoding="utf-8") as of:
#             json.dump(result, of, ensure_ascii=False)
#     else:
#         print(result['err_msg'], result['err_no'])

In [10]:
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Oct/2019 23:01:59] "POST / HTTP/1.1" 200 -


In [3]:
def convert_aac_to_wav(aacPath):
    # 還在思考路徑怎麼寫...
    # convert aac to wav
    aac = AudioSegment.from_file(aacPath)
    aac.export(aacPath.split(".")[-2].replace("/","") + ".wav", format="wav")

    # check data
    voice = wave.open(aacPath.split(".")[-2].replace("/","") + ".wav", "rb")
    channel, _, framerate, _, _, _ = voice.getparams()
    print(f"###開始轉檔###\n聲道數:{channel} 音頻:{framerate}\n###轉檔成功###")

In [4]:
accPath = "./audios/10832336465039.aac"
convert_aac_to_wav(accPath)

###開始轉檔###
聲道數:1 音頻:16000
###轉檔成功###


In [6]:
from aip import AipSpeech
import json
import time

""" 你的 APPID AK SK """
APP_ID = '17610496'
API_KEY = '3HxD7AjTwLrjEdWzjno97lbl'
SECRET_KEY = 'rPBbzb3enl2EA0cK0n98zHktbWDqYPt8'

begin = time.time()
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)


# 读取文件
def get_file_content(filepath):
    with open(filepath, 'rb') as fp:
        return fp.read()


# 识别本地文件
voicefile = "audios10832336465039.wav"
# base = "./audios/"
# print(base+voicefile)
result = client.asr(get_file_content(voicefile), 'wav', 16000, {
    'dev_pid': 1536,
})
# print(result)
print("Request time cost %f" % (time.time() - begin))
if result['err_no'] == 0:
    ofile = "result_%s.txt" % (voicefile)
    with open(ofile, "w", encoding="utf-8") as of:
        json.dump(result, of, ensure_ascii=False)
else:
    print(result['err_msg'], result['err_no'])

print("完成了")

Request time cost 3.283937
完成了


In [ ]:
# import wave

# with wave.open('./audios/10826265961358.wav', "rb") as f:
#     f = wave.open('./audios/10826265961358.wav')
#     print(f.getparams())

In [ ]:
# import librosa
# import soundfile as sf
# x,_ = librosa.load('./audios/10826265961358.wav', sr=16000)
# sf.write('tmp.wav', x, 16000)
# wave.open('tmp.wav','r')

In [ ]:
# import os
# import wave
# os.system(r"ffmpeg -i 10826265961358.wav b0.wav")
# wave.open('b0', "r")

In [ ]:
# b=os.popen(r"ffmpeg -y -i ./audios/10826265961358.wav  -acodec pcm_s16le -f s16le -ac 1 -ar 16000 ./audios/1082626596135800.wav")